In [1]:
#set directory path for dataset
import os
os.chdir(r"C:\Users\ASUS\IA-MASKdetection\Data")
Dataset="dataset"
Data_dir=os.listdir(Dataset)
print(Data_dir)

['without_mask', 'with_mask']


In [2]:
#import libraries
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from skimage import io
img_rows,img_cols = 112,112

images = []
labels = []

for category in Data_dir:
    folder_path = os.path.join(Dataset,category)
    for img in os.listdir(folder_path):
        img_path= os.path.join(folder_path,img)
        #img=cv2.imread(img_path)
        img = io.imread(img_path)
        
        try:
            #convert the image into gray scale
            grayscale_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
           
            #resising the gray scaled image into size 56*56 in order to keep size of  image consistent
            resized_image = cv2.resize(grayscale_img,(img_rows,img_cols))
            images.append(resized_image)
            labels.append(category)
        #exception handling in case any error occurs
        except Exception as e:
            print("Exception : ",e)

images=np.array(images)/255.0
images=np.reshape(images,(images.shape[0],img_rows,img_cols,1))

#perform one hot encoding on the labels since the label are in textual form
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)
labels = np.array(labels)

(train_X,test_X,train_Y,test_Y) = train_test_split(images,labels,test_size=0.25,random_state=0)

            

In [4]:
#import necesary liberies keras
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D, MaxPooling2D

#Define model parametres
num_classes = 2
batch_size=32

#build CNN model using Sequential API
model=Sequential()

#First layer group containing Convolution,relu and MaxPooling layers 
# 1 define the channel of the image, in this case it's  grayscale image, otherwise we put 3
model.add(Conv2D(64,(3,3),input_shape=(img_rows,img_cols,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


#Second layer group containing Convolution,relu and MaxPooling layers
model.add(Conv2D(128,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))



#Flatten and dropout layer to stack the output convolutions above as well as after overfitting
model.add(Flatten())
model.add(Dropout(0.5))


#softmax classifier
model.add(Dense(64,activation='relu'))
model.add(Dense(num_classes,activation='softmax'))


print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 110, 110, 64)      640       
_________________________________________________________________
activation (Activation)      (None, 110, 110, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 55, 55, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 53, 53, 128)       73856     
_________________________________________________________________
activation_1 (Activation)    (None, 53, 53, 128)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 26, 26, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 86528)             0

In [5]:
from keras.optimizers import Adam

epochs = 50
model.compile(loss='categorical_crossentropy',optimizer = Adam(lr=0.001), metrics= ['accuracy'])

fitted_model = model.fit(train_X,train_Y, epochs=epochs, validation_split = 0.25 ) 


Epoch 1/50
68/68 [==============================] - 63s 927ms/step - loss: 0.6337 - accuracy: 0.6849 - val_loss: 0.5179 - val_accuracy: 0.7705
Epoch 2/50
68/68 [==============================] - 60s 883ms/step - loss: 0.3653 - accuracy: 0.8418 - val_loss: 0.3720 - val_accuracy: 0.8401
Epoch 3/50
68/68 [==============================] - 61s 899ms/step - loss: 0.2481 - accuracy: 0.8970 - val_loss: 0.3349 - val_accuracy: 0.8512
Epoch 4/50
68/68 [==============================] - 61s 896ms/step - loss: 0.1904 - accuracy: 0.9299 - val_loss: 0.3061 - val_accuracy: 0.8748
Epoch 5/50
68/68 [==============================] - 62s 910ms/step - loss: 0.1430 - accuracy: 0.9420 - val_loss: 0.3290 - val_accuracy: 0.8860
Epoch 6/50
68/68 [==============================] - 60s 888ms/step - loss: 0.0958 - accuracy: 0.9652 - val_loss: 0.3650 - val_accuracy: 0.8901
Epoch 7/50
68/68 [==============================] - 61s 891ms/step - loss: 0.0778 - accuracy: 0.9703 - val_loss: 0.4160 - val_accuracy: 0.8873

In [15]:
model.save('C:/Users/ASUS/IA-MASKdetection/face_mask_detection_alert_system.h5')